In [ ]:
import numpy as np
import scipy.stats as sc
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output
from time import sleep
import pandas as pd
import math
import random

In [ ]:
def plot_bar_3(susceptible, infected, vaccinated, recovered,population):
  N=4
  ind=np.arange(N)
  width=0.3

  
  fig = plt.subplots(figsize =(10, 7)) 
  bars = np.add(susceptible, infected).tolist()
  bars2=np.add(bars, vaccinated).tolist()

  p1 = plt.bar(ind, susceptible, width) 
  p2 = plt.bar(ind, infected,width, bottom=susceptible) 
  p3 = plt.bar(ind, vaccinated,width, bottom=bars) 
  p4 = plt.bar(ind, recovered ,width, bottom=bars2) 
  
  plt.ylabel('Headcount') 
  plt.title('Animation showing the change in number of people in each subclass overtime') 
  plt.xticks(ind, ('Medical', 'High-Risk', 'Essential', 'Low-Risk')) 
  plt.yticks(np.arange(0, population/4, population/4/10)) 
  plt.legend((p1[0], p2[0], p3[0], p4[0]), ('susceptible', 'infected','vaccinated','recovered')) 
  
  plt.show()

In [ ]:
k=3
R_0=1.5

In [ ]:
def run_SIR_clusters_2(k_high,k_low,n,R_0,p_i,p_vac,p_dosage):
    """
    Run SIR simulation
    ----------
    k_high: (int) number of average meetings (higher)
    k_low: (int) number of average meetings (lower)
    n: number of people in the entire populatin (n/4 per cluster)
    R_0: basic reproduction number (number of people you can infect if you are infected)
    p_i: probability of infected with symptoms (array)
    p_vac: probability of each class receiving a vaccine (array)
    p_dosage: percentage of the total population that recives the vaccine daily
    Returns
    ---------
    nodes_R: Number of people who got infected and recovered
    """
    

    medical_infected = random.sample(range(int(0),int(n/4)),500) # infected medical worker
    essential_infected = random.sample(range(int(n/4),int(n/2)),2500) # infected essential worker
    high_risk_infected = random.sample(range(int(n/2),int(3*n/4)),2500) # infected high-risk person
    low_risk_infected = random.sample(range(int(3*n/4),int(n)),2500) # infected low-risk preson
    nodes_S_medical = np.arange(0,n/4).tolist() # remove infected agent from cluster


    medical_vac = []
    essential_vac = []
    high_risk_vac = []
    low_risk_vac = []

    day = 0 

    num_dosage = p_dosage*n # number of vaccine dosage available daily
    leftover_vaccine = 0 # number of leftover vaccine 

    for each in medical_infected:
      nodes_S_medical.remove(each)
    nodes_S_essential = np.arange(n/4,n/2).tolist() # remove infected agent from cluster

    for each in essential_infected:
      nodes_S_essential.remove(each)
    
    nodes_S_high_risk = np.arange(n/2,3*n/4).tolist() # remove infected agent from cluster

    for each in high_risk_infected:
      nodes_S_high_risk.remove(each)
    nodes_S_low_risk = np.arange(3*n/4,n).tolist() # remove infected agent from cluster

    for each in low_risk_infected:
      nodes_S_low_risk.remove(each)

    nodes_I_medical = {} # dictionary in the format of (index,'A'/'S') with A being asymptomatic and S being symptomatic
    nodes_I_essential = {}
    nodes_I_high_risk = {}
    nodes_I_low_risk = {}

    for each in medical_infected:
      nodes_I_medical[each] = 'S'

    for each in essential_infected:
      nodes_I_essential[each] = 'S'
    
    for each in high_risk_infected:
      nodes_I_high_risk[each] = 'S'

    for each in low_risk_infected:
      nodes_I_low_risk[each] = 'S'


    nodes_R_medical = [] # recovered medical 
    nodes_R_high_risk = [] # recovered high risk
    nodes_R_essential = [] # recovered essential
    nodes_R_low_risk = [] # recovered low risk

    h = k_high/(n-1)
    l = k_low/(n-1)


    beta = (k_high+k_low)/2/R_0 # recovery rate (we took the average of k_high and k_low)

    lambda_matrix = [[h,h,l,l],[l,h,l,h],[l,h,h,h],[l,l,l,h],[h,h,h,h]]
    
    total_11 = len(nodes_S_medical)*sum(value == 'A' for value in nodes_I_medical.values())*lambda_matrix[0][0] # susceptible medical meets asymptomatic infected medical
    total_12 = len(nodes_S_medical)*sum(value == 'A' for value in nodes_I_essential.values())*lambda_matrix[0][1] # susceptible medical meets asymptomatic infected essential
    total_13 = len(nodes_S_medical)*sum(value == 'A' for value in nodes_I_high_risk.values())*lambda_matrix[0][2] # susceptible medical meets asymptomatic infected high-risk
    total_14 = len(nodes_S_medical)*sum(value == 'A' for value in nodes_I_low_risk.values())*lambda_matrix[0][3] # susceptible medical meets asymptomatic infected low-risk
    total_21 = len(nodes_S_essential)*sum(value == 'A' for value in nodes_I_medical.values())*lambda_matrix[1][0] # susceptible essential meets asymptomatic infected medical
    total_22 = len(nodes_S_essential)*sum(value == 'A' for value in nodes_I_essential.values())*lambda_matrix[1][1] # susceptible essential meets asymptomatic infected essential
    total_23 = len(nodes_S_essential)*sum(value == 'A' for value in nodes_I_high_risk.values())*lambda_matrix[1][2] # susceptible essential meets asymptomatic infected high risk
    total_24 = len(nodes_S_essential)*sum(value == 'A' for value in nodes_I_low_risk.values())*lambda_matrix[1][3] # susceptible essential meets asymptomatic infected low risk
    total_31 = len(nodes_S_high_risk)*sum(value == 'A' for value in nodes_I_medical.values())*lambda_matrix[2][0] # susceptible high risk meets asymptomatic infected medical
    total_32 = len(nodes_S_high_risk)*sum(value == 'A' for value in nodes_I_essential.values())*lambda_matrix[2][1] # susceptible high risk meets infected essential
    total_33 = len(nodes_S_high_risk)*sum(value == 'A' for value in nodes_I_high_risk.values())*lambda_matrix[2][2] # susceptible high risk meets infected high risk
    total_34 = len(nodes_S_high_risk)*sum(value == 'A' for value in nodes_I_low_risk.values())*lambda_matrix[2][3] # susceptible high risk meets infected low risk
    total_41 = len(nodes_S_low_risk)*sum(value == 'A' for value in nodes_I_medical.values())*lambda_matrix[3][0] # susceptible low risk meets infected medical
    total_42 = len(nodes_S_low_risk)*sum(value == 'A' for value in nodes_I_essential.values())*lambda_matrix[3][1] # susceptible low risk meets infected essential
    total_43 = len(nodes_S_low_risk)*sum(value == 'A' for value in nodes_I_high_risk.values())*lambda_matrix[3][2] # susceptible low risk meets infected high risk
    total_44 = len(nodes_S_low_risk)*sum(value == 'A' for value in nodes_I_low_risk.values())*lambda_matrix[3][3] # susceptible low risk meets infected low risk

    #infected agents from all four classes can meet suscpetible medical workers in C1
    total_11s = len(nodes_S_medical)*sum(value == 'S' for value in nodes_I_medical.values())*lambda_matrix[4][0]
    total_12s = len(nodes_S_medical)*sum(value == 'S' for value in nodes_I_essential.values())*lambda_matrix[4][1]
    total_13s = len(nodes_S_medical)*sum(value == 'S' for value in nodes_I_high_risk.values())*lambda_matrix[4][2]
    total_14s = len(nodes_S_medical)*sum(value == 'S' for value in nodes_I_low_risk.values())*lambda_matrix[4][3]
    
    rate_recover = beta*(len(nodes_I_medical) + len(nodes_I_essential) + len(nodes_I_high_risk) + len(nodes_I_low_risk))
    rate_total = total_11 + total_12 + total_13 + total_14 + total_21 + total_22 + total_23 + total_24 + total_31 + total_32 + total_33 + total_34 + total_41 + total_42 + total_43 + total_44 + rate_recover +total_11s + total_12s+ total_13s+ total_14s
    t = 0
    times = []
    nextEvent = t + np.random.exponential(1/rate_total)

    counter = 0
    num_vaccine = [int(num_dosage*p_vac[0]),int(num_dosage*p_vac[1]),int(num_dosage*p_vac[2]),int(num_dosage*p_vac[3])]
    leftover_vaccine = 0

    while (len(nodes_S_medical)!=0 or len(nodes_S_essential)!=0 or len(nodes_S_high_risk)!=0 or len(nodes_S_low_risk)!=0) and (len(nodes_I_medical)!=0 or len(nodes_I_essential)!=0 or len(nodes_I_high_risk)!=0 or len(nodes_I_low_risk)!=0): # no more susceptible or infected individuals

        counter = counter + 1
        t = t + nextEvent
        times.append(t)

        #check to see if it is a new day. if so, make more vaccines available
        while t - day >= 1:
            day = day + 1
            num_vaccine = [int(num_dosage*p_vac[0]),int(num_dosage*p_vac[1]),int(num_dosage*p_vac[2]),int(num_dosage*p_vac[3])] # number of vaccine for each class in an array format

        #Simplifying assumption: when vaccines are made available, susceptible 
        #agents will immediately claim these vaccines before interacting with
        #any other agents.

        
        num_vaccine[0] = num_vaccine[0] + leftover_vaccine
        leftover_vaccine = 0

        # first order: distribution of vaccine to medical 
        if len(nodes_S_medical) != 0 :
          # if there are more infected medical workers than vaccine
          if len(nodes_S_medical) >= num_vaccine[0]:
            for each in range(num_vaccine[0]):
              i = nodes_S_medical[0]
              medical_vac.append(i)
              nodes_S_medical.remove(i)
            num_vaccine[0] = 0

          # if there are more vaccines than there are infected medical workers
          else:
            for each in range(len(nodes_S_medical)):
              i = nodes_S_medical[0]
              medical_vac.append(i)
              nodes_S_medical.remove(i)
            num_vaccine[1] = num_vaccine[1] + (num_vaccine[0]-len(nodes_S_medical))
            num_vaccine[0] = 0

        # second order: distribution of vaccine to high risk 
        if len(nodes_S_high_risk)!=0:
          # if there are more infected high risk individuals than vaccine
          if len(nodes_S_high_risk) >= num_vaccine[1]:
            for each in range(num_vaccine[1]):
              i = nodes_S_high_risk[0]
              high_risk_vac.append(i)
              nodes_S_high_risk.remove(i)
            num_vaccine[1] = 0

          # if there are more vaccines than there are infected medical workers
          else:
            for each in range(len(nodes_S_high_risk)):
              i = nodes_S_high_risk[0]
              high_risk_vac.append(i)
              nodes_S_high_risk.remove(i)
            num_vaccine[2] = num_vaccine[2] + (num_vaccine[1]-len(nodes_S_high_risk))
            num_vaccine[1] = 0

        # third order: distribution of vaccine to essential
        if len(nodes_S_essential)!=0:
          # if there are more infected essential individuals than vaccine
          if len(nodes_S_essential) >= num_vaccine[2]:
            for each in range(num_vaccine[2]):
              i = nodes_S_essential[0]
              essential_vac.append(i)
              nodes_S_essential.remove(i)
            num_vaccine[2] = 0

          # if there are more vaccines than there are infected essential workers
          else:
            for each in range(len(nodes_S_high_risk)):
              i = nodes_S_high_risk[0]
              essential_vac.append(i)
              nodes_S_essential.remove(i)
            num_vaccine[3] = num_vaccine[3] + (num_vaccine[2]-len(nodes_S_essential))
            num_vaccine[2] = 0

        # fourth order: distribution of vaccine to low risk
        if len(nodes_S_low_risk)!=0:
          # if there are more infected essential individuals than vaccine
          if len(nodes_S_low_risk) >= num_vaccine[3]:
            for each in range(num_vaccine[3]):
              i = nodes_S_low_risk[0]
              low_risk_vac.append(i)
              nodes_S_low_risk.remove(i)
            num_vaccine[3] = 0

          # if there are more vaccines than there are infected essential workers
          else:
            for each in range(len(nodes_S_low_risk)):
              i = nodes_S_low_risk[0]
              low_risk_vac.append(i)
              nodes_S_low_risk.remove(i)
            leftover_vaccine = num_vaccine[3]-len(nodes_S_low_risk)
            num_vaccine[3] = 0

        U = np.random.rand()

        events = [0, total_11/rate_total, (total_11+total_12)/rate_total, (total_11+total_12+total_13)/rate_total, 
                  (total_11+total_12+total_13)/rate_total, (total_11+total_12+total_13+total_14)/rate_total,
                  (total_11+total_12+total_13+total_14+total_21)/rate_total,(total_11+total_12+total_13+total_14+total_21+total_22)/rate_total,
                  (total_11+total_12+total_13+total_14+total_21+total_22 + total_23)/rate_total, 
                  (total_11+total_12+total_13+total_14+total_21+total_22 + total_23+total_24)/rate_total,
                  (total_11+total_12+total_13+total_14+total_21+total_22 + total_23+total_24+total_31)/rate_total,
                  (total_11+total_12+total_13+total_14+total_21+total_22 + total_23+total_24+total_31+total_32)/rate_total,
                  (total_11+total_12+total_13+total_14+total_21+total_22 + total_23+total_24+total_31+total_32+total_33)/rate_total,
                  (total_11+total_12+total_13+total_14+total_21+total_22 + total_23+total_24+total_31+total_32+total_33+total_34)/rate_total,
                  (total_11+total_12+total_13+total_14+total_21+total_22 + total_23+total_24+total_31+total_32+total_33+total_34+total_41)/rate_total,
                  (total_11+total_12+total_13+total_14+total_21+total_22 + total_23+total_24+total_31+total_32+total_33+total_34+total_41+total_42)/rate_total,
                  (total_11+total_12+total_13+total_14+total_21+total_22 + total_23+total_24+total_31+total_32+total_33+total_34+total_41+total_42+total_43)/rate_total,
                  (total_11+total_12+total_13+total_14+total_21+total_22 + total_23+total_24+total_31+total_32+total_33+total_34+total_41+total_42+total_43+total_44)/rate_total,
                  (total_11+total_12+total_13+total_14+total_21+total_22 + total_23+total_24+total_31+total_32+total_33+total_34+total_41+total_42+total_43+total_44+total_11s)/rate_total,
                  (total_11+total_12+total_13+total_14+total_21+total_22 + total_23+total_24+total_31+total_32+total_33+total_34+total_41+total_42+total_43+total_44+total_11s+total_12s)/rate_total,
                  (total_11+total_12+total_13+total_14+total_21+total_22 + total_23+total_24+total_31+total_32+total_33+total_34+total_41+total_42+total_43+total_44+total_11s+total_12s+total_13s)/rate_total,
                  (total_11+total_12+total_13+total_14+total_21+total_22 + total_23+total_24+total_31+total_32+total_33+total_34+total_41+total_42+total_43+total_44+total_11s+total_12s+total_13s+total_14s)/rate_total]
        

        # scenario 1: susceptible medical meets infected medical   # find a random susceptible medical to infect
        if events[0]<= U<= events[1] and len(nodes_S_medical)!=0:
            i=np.random.choice(nodes_S_medical)
           # determine whether symptomatic or asymptomatic
            p_symptom = np.random.uniform(0,1)
            if p_symptom <= p_i[0]: # symptomatic
                nodes_I_medical[i] = 'S'
                nodes_S_medical.remove(i)
            else: # asymptomatic
                nodes_I_medical[i] = 'A'
                nodes_S_medical.remove(i)
        # scenario 2: susceptible medical meets infected essential
        elif U <= events[2] and len(nodes_S_medical)!=0:
            i = np.random.choice(nodes_S_medical)
            # determine whether symptomatic or asymptomatic
            p_symptom = np.random.uniform(0,1)
            if p_symptom <= p_i[0]: # symptomatic
                nodes_I_medical[i] = 'S'
                nodes_S_medical.remove(i)
            else: # asymptomatic
                nodes_I_medical[i] = 'A'
                nodes_S_medical.remove(i)
        # scenario 3: susceptible medical meets infected high risk
        elif U <=events[3] and len(nodes_S_medical)!=0:
            i = np.random.choice(nodes_S_medical)
            # determine whether symptomatic or asymptomatic
            p_symptom = np.random.uniform(0,1)
            if p_symptom <= p_i[0]: # symptomatic
                nodes_I_medical[i] = 'S'
                nodes_S_medical.remove(i)
            else: # asymptomatic
                nodes_I_medical[i] = 'A'
                nodes_S_medical.remove(i)
        # scenario 4: susceptible medical meets infected low risk
        elif U<=events[4] and len(nodes_S_medical)!=0:
            i = np.random.choice(nodes_S_medical)
            # determine whether symptomatic or asymptomatic
            p_symptom = np.random.uniform(0,1)
            if p_symptom <= p_i[0]: # symptomatic
                nodes_I_medical[i] = 'S'
                nodes_S_medical.remove(i)
            else: # asymptomatic
                nodes_I_medical[i] = 'A'
                nodes_S_medical.remove(i)
        # scenario 5: susceptible essential meets infected medical
        elif U<=events[5] and len(nodes_S_essential)!=0:
            i = np.random.choice(nodes_S_essential)
            # determine whether symptomatic or asymptomatic
            p_symptom = np.random.uniform(0,1)
            if p_symptom <= p_i[1]: # symptomatic
                nodes_I_essential[i] = 'S'
                nodes_S_essential.remove(i)
            else: # asymptomatic
                nodes_I_essential[i] = 'A'
                nodes_S_essential.remove(i)
        # scenario 6: susceptible essential meets infected essential
        elif U<=events[6] and len(nodes_S_essential)!=0:
            i = np.random.choice(nodes_S_essential)
            # determine whether symptomatic or asymptomatic
            p_symptom = np.random.uniform(0,1)
            if p_symptom <= p_i[1]: # symptomatic
                nodes_I_essential[i] = 'S'
                nodes_S_essential.remove(i)
            else: # asymptomatic
                nodes_I_essential[i] = 'A'
                nodes_S_essential.remove(i)
        # scenario 7: susceptible essential meets infected high risk
        elif U<=events[7] and len(nodes_S_essential)!=0:
            i = np.random.choice(nodes_S_essential)
            # determine whether symptomatic or asymptomatic
            p_symptom = np.random.uniform(0,1)
            if p_symptom <= p_i[1]: # symptomatic
                nodes_I_essential[i] = 'S'
                nodes_S_essential.remove(i)
            else: # asymptomatic
                nodes_I_essential[i] = 'A'
                nodes_S_essential.remove(i)
        # scenario 8: susceptible essential meets infected low risk
        elif U<=events[8] and len(nodes_S_essential)!=0:
            i = np.random.choice(nodes_S_essential)
            # determine whether symptomatic or asymptomatic
            p_symptom = np.random.uniform(0,1)
            if p_symptom <= p_i[1]: # symptomatic
                nodes_I_essential[i] = 'S'
                nodes_S_essential.remove(i)
            else: # asymptomatic
                nodes_I_essential[i] = 'A'
                nodes_S_essential.remove(i)
        # scenario 9: susceptible high risk meets infected medical
        elif U<=events[9] and len(nodes_S_high_risk)!=0:
            i = np.random.choice(nodes_S_high_risk)
            # determine whether symptomatic or asymptomatic
            p_symptom = np.random.uniform(0,1)
            if p_symptom <= p_i[2]: # symptomatic
                nodes_I_high_risk[i] = 'S'
                nodes_S_high_risk.remove(i)
            else: # asymptomatic
                nodes_I_high_risk[i] = 'A'
                nodes_S_high_risk.remove(i)
        # scenario 10: susceptible high risk meets infected essential
        elif U<=events[10] and len(nodes_S_high_risk)!=0:
            i = np.random.choice(nodes_S_high_risk)
            # determine whether symptomatic or asymptomatic
            p_symptom = np.random.uniform(0,1)
            if p_symptom <= p_i[2]: # symptomatic
                nodes_I_high_risk[i] = 'S'
                nodes_S_high_risk.remove(i)
            else: # asymptomatic
                nodes_I_high_risk[i] = 'A'
                nodes_S_high_risk.remove(i)
        # scenario 11: susceptible high risk meets infected high risk
        elif U<=events[11] and len(nodes_S_high_risk)!=0:
            i = np.random.choice(nodes_S_high_risk)
            # determine whether symptomatic or asymptomatic
            p_symptom = np.random.uniform(0,1)
            if p_symptom <= p_i[2]: # symptomatic
                nodes_I_high_risk[i] = 'S'
                nodes_S_high_risk.remove(i)
            else: # asymptomatic
                nodes_I_high_risk[i] = 'A'
                nodes_S_high_risk.remove(i)
        # scenario 12: susceptible high risk meets infected low risk
        elif U<=events[12] and len(nodes_S_high_risk)!=0:
            i = np.random.choice(nodes_S_high_risk)
            # determine whether symptomatic or asymptomatic
            p_symptom = np.random.uniform(0,1)
            if p_symptom <= p_i[2]: # symptomatic
                nodes_I_high_risk[i] = 'S'
                nodes_S_high_risk.remove(i)
            else: # asymptomatic
                nodes_I_high_risk[i] = 'A'
                nodes_S_high_risk.remove(i)
        # scenario 13: susceptible low risk meets infected medical
        elif U<=events[13] and len(nodes_S_low_risk)!=0:
            i = np.random.choice(nodes_S_low_risk)
            # determine whether symptomatic or asymptomatic
            p_symptom = np.random.uniform(0,1)
            if p_symptom <= p_i[3]: # symptomatic
                nodes_I_low_risk[i] = 'S'
                nodes_S_low_risk.remove(i)
            else: # asymptomatic
                nodes_I_low_risk[i] = 'A'
                nodes_S_low_risk.remove(i)
        # scenario 14: susceptible low risk meets infected essential
        elif U<=events[14] and len(nodes_S_low_risk)!=0:
            i = np.random.choice(nodes_S_low_risk)
            # determine whether symptomatic or asymptomatic
            p_symptom = np.random.uniform(0,1)
            if p_symptom <= p_i[3]: # symptomatic
                nodes_I_low_risk[i] = 'S'
                nodes_S_low_risk.remove(i)
            else: # asymptomatic
                nodes_I_low_risk[i] = 'A'
                nodes_S_low_risk.remove(i)
        # scenario 15: susceptible low risk meets infected high risk
        elif U<=events[15] and len(nodes_S_low_risk)!=0:
            i = np.random.choice(nodes_S_low_risk)
            # determine whether symptomatic or asymptomatic
            p_symptom = np.random.uniform(0,1)
            if p_symptom <= p_i[3]: # symptomatic
                nodes_I_low_risk[i] = 'S'
                nodes_S_low_risk.remove(i)
            else: # asymptomatic
                nodes_I_low_risk[i] = 'A'
                nodes_S_low_risk.remove(i)
        # scenario 16: susceptible low risk meets infected low risk
        elif U<=events[16] and len(nodes_S_low_risk)!=0:
            i = np.random.choice(nodes_S_low_risk)
            # determine whether symptomatic or asymptomatic
            p_symptom = np.random.uniform(0,1)
            if p_symptom <= p_i[3]: # symptomatic
                nodes_I_low_risk[i] = 'S'
                nodes_S_low_risk.remove(i)
            else: # asymptomatic
                nodes_I_low_risk[i] = 'A'
                nodes_S_low_risk.remove(i)
        # scenario 17: susceptible medical meets symptomatic infected medical
        elif U<=events[17] and len(nodes_S_medical)!=0:
            i = np.random.choice(nodes_S_medical)
            # determine whether symptomatic or asymptomatic
            p_symptom = np.random.uniform(0,1)
            if p_symptom <= p_i[3]: # symptomatic
                nodes_I_medical[i] = 'S'
                nodes_S_medical.remove(i)
            else: # asymptomatic
                nodes_I_medical[i] = 'A'
                nodes_S_medical.remove(i)
        # recovery
        # scenario 18: susceptible medical meets symptomatic infected essential workers
        elif U<=events[18] and len(nodes_S_medical)!=0:
            i = np.random.choice(nodes_S_medical)
            # determine whether symptomatic or asymptomatic
            p_symptom = np.random.uniform(0,1)
            if p_symptom <= p_i[3]: # symptomatic
                nodes_I_medical[i] = 'S'
                nodes_S_medical.remove(i)
            else: # asymptomatic
                nodes_I_medical[i] = 'A'
                nodes_S_medical.remove(i)
        # scenario 19: susceptible medical meets symptomatic infected high risk workers
        elif U<=events[19] and len(nodes_S_medical)!=0:
            i = np.random.choice(nodes_S_medical)
            # determine whether symptomatic or asymptomatic
            p_symptom = np.random.uniform(0,1)
            if p_symptom <= p_i[3]: # symptomatic
                nodes_I_medical[i] = 'S'
                nodes_S_medical.remove(i)
            else: # asymptomatic
                nodes_I_medical[i] = 'A'
                nodes_S_medical.remove(i)
        # scenario 20: susceptible medical meets symptomatic infected low risk workers
        elif U<=events[20] and len(nodes_S_medical)!=0:
            i = np.random.choice(nodes_S_medical)
            # determine whether symptomatic or asymptomatic
            p_symptom = np.random.uniform(0,1)
            if p_symptom <= p_i[3]: # symptomatic
                nodes_I_medical[i] = 'S'
                nodes_S_medical.remove(i)
            else: # asymptomatic
                nodes_I_medical[i] = 'A'
                nodes_S_medical.remove(i)
        # recovery
        else:
  
            i = np.random.choice(list(nodes_I_medical.keys()) + list(nodes_I_essential.keys()) + list(nodes_I_high_risk.keys()) + list(nodes_I_low_risk.keys())) # choose random infected person to recover

            # find out which class the recovered agent is in
            if i in nodes_I_medical.keys():
                nodes_R_medical.append(i)
                del nodes_I_medical[i]
            elif i in nodes_I_essential.keys():
                nodes_R_essential.append(i)
                del nodes_I_essential[i]
            elif i in nodes_I_high_risk.keys():
                nodes_R_high_risk.append(i)
                del nodes_I_high_risk[i]
            else:
                nodes_R_low_risk.append(i)
                del nodes_I_low_risk[i]

        

        total_11 = len(nodes_S_medical)*sum(value == 'A' for value in nodes_I_medical.values())*lambda_matrix[0][0] # susceptible medical meets asymptomatic infected medical
        total_12 = len(nodes_S_medical)*sum(value == 'A' for value in nodes_I_essential.values())*lambda_matrix[0][1] # susceptible medical meets asymptomatic infected essential
        total_13 = len(nodes_S_medical)*sum(value == 'A' for value in nodes_I_high_risk.values())*lambda_matrix[0][2] # susceptible medical meets asymptomatic infected high-risk
        total_14 = len(nodes_S_medical)*sum(value == 'A' for value in nodes_I_low_risk.values())*lambda_matrix[0][3] # susceptible medical meets asymptomatic infected low-risk
        total_21 = len(nodes_S_essential)*sum(value == 'A' for value in nodes_I_medical.values())*lambda_matrix[1][0] # susceptible essential meets asymptomatic infected medical
        total_22 = len(nodes_S_essential)*sum(value == 'A' for value in nodes_I_essential.values())*lambda_matrix[1][1] # susceptible essential meets asymptomatic infected essential
        total_23 = len(nodes_S_essential)*sum(value == 'A' for value in nodes_I_high_risk.values())*lambda_matrix[1][2] # susceptible essential meets asymptomatic infected high risk
        total_24 = len(nodes_S_essential)*sum(value == 'A' for value in nodes_I_low_risk.values())*lambda_matrix[1][3] # susceptible essential meets asymptomatic infected low risk
        total_31 = len(nodes_S_high_risk)*sum(value == 'A' for value in nodes_I_medical.values())*lambda_matrix[2][0] # susceptible high risk meets asymptomatic infected medical
        total_32 = len(nodes_S_high_risk)*sum(value == 'A' for value in nodes_I_essential.values())*lambda_matrix[2][1] # susceptible high risk meets infected essential
        total_33 = len(nodes_S_high_risk)*sum(value == 'A' for value in nodes_I_high_risk.values())*lambda_matrix[2][2] # susceptible high risk meets infected high risk
        total_34 = len(nodes_S_high_risk)*sum(value == 'A' for value in nodes_I_low_risk.values())*lambda_matrix[2][3] # susceptible high risk meets infected low risk
        total_41 = len(nodes_S_low_risk)*sum(value == 'A' for value in nodes_I_medical.values())*lambda_matrix[3][0] # susceptible low risk meets infected medical
        total_42 = len(nodes_S_low_risk)*sum(value == 'A' for value in nodes_I_essential.values())*lambda_matrix[3][1] # susceptible low risk meets infected essential
        total_43 = len(nodes_S_low_risk)*sum(value == 'A' for value in nodes_I_high_risk.values())*lambda_matrix[3][2] # susceptible low risk meets infected high risk
        total_44 = len(nodes_S_low_risk)*sum(value == 'A' for value in nodes_I_low_risk.values())*lambda_matrix[3][3] # susceptible low risk meets infected low risk

        total_11s = len(nodes_S_medical)*sum(value == 'S' for value in nodes_I_medical.values())*lambda_matrix[4][0]
        total_12s = len(nodes_S_medical)*sum(value == 'S' for value in nodes_I_essential.values())*lambda_matrix[4][1]
        total_13s = len(nodes_S_medical)*sum(value == 'S' for value in nodes_I_high_risk.values())*lambda_matrix[4][2]
        total_14s = len(nodes_S_medical)*sum(value == 'S' for value in nodes_I_low_risk.values())*lambda_matrix[4][3]

        rate_recover = beta*(len(nodes_I_medical) + len(nodes_I_essential) + len(nodes_I_high_risk) + len(nodes_I_low_risk))
        rate_total = total_11 + total_12 + total_13 + total_14 + total_21 + total_22 + total_23 + total_24 + total_31 + total_32 + total_33 + total_34 + total_41 + total_42 + total_43 + total_44 + rate_recover +total_11s + total_12s+ total_13s+ total_14s

        if rate_total>0:
          nextEvent= np.random.exponential(1/rate_total)


        #R_0 = R_0 * (len(nodes_S_medical)+len(nodes_S_essential)+len(nodes_S_high_risk)+len(nodes_S_low_risk))
        #beta = (k_high+k_low)/2/R_0

        # order: S, I, R, V
        medical_len = [len(nodes_S_medical),len(nodes_I_medical),len(nodes_R_medical),len(medical_vac)]
        essential_len = [len(nodes_S_essential),len(nodes_I_essential),len(nodes_R_essential),len(essential_vac)]
        high_risk_len = [len(nodes_S_high_risk),len(nodes_I_high_risk),len(nodes_R_high_risk),len(high_risk_vac)]
        low_risk_len = [len(nodes_S_low_risk),len(nodes_I_low_risk),len(nodes_R_low_risk),len(low_risk_vac)]

        susceptible = [len(nodes_S_medical),len(nodes_S_high_risk),len(nodes_S_essential),len(nodes_S_low_risk)]
        infected = [len(nodes_I_medical),len(nodes_I_high_risk),len(nodes_I_essential),len(nodes_I_low_risk)]
        vaccinated = [len(medical_vac),len(high_risk_vac),len(essential_vac),len(low_risk_vac)]
        recovered = [len(nodes_R_medical),len(nodes_R_high_risk),len(nodes_R_essential),len(nodes_R_low_risk)]

        
        #plot_bar_3(susceptible, infected, vaccinated, recovered,n)
        #plt.axis('equal')
        #plt.axis('off')
        #plt.show()
        #sleep(0.02)
        #clear_output(wait=True)
        

    return medical_len,high_risk_len,essential_len,low_risk_len,day

In [ ]:
# Model 1A: no social distancing, 1% dosage, equal distribution 

infection = []
days = []

num_trials = 10
for each in range(num_trials):  
    medical_len, high_risk_len, essential_len, low_risk_len, day = run_SIR_clusters_2(10,4,100000,1.5,[0,0,0,0],[0.25,0.25,0.25,0.25],0.01)
    infection.append(medical_len[1]+high_risk_len[1]+essential_len[1]+low_risk_len[1])
    days.append(day)

LB_infection = np.mean(infection) - scipy.stats.norm.ppf(1-0.05/2)*np.sqrt(np.var(infection)/num_trials) 
UB_infection = np.mean(infection) + scipy.stats.norm.ppf(1-0.05/2)*np.sqrt(np.var(infection)/num_trials)

LB_days = np.mean(days) - scipy.stats.norm.ppf(1-0.05/2)*np.sqrt(np.var(days)/num_trials)
UB_days = np.mean(days) + scipu.stats.norm.ppf(1-0.05/2)*np.sqrt(np.var(days)/num_trials)

print('The 95% CI for the number of infected people is,(', LB_infection,', ', UB_infection, ')')
print('The 95% CI for the number of days it takes to contain the virus is (', LB_days,', ', UB_days, ')')

In [ ]:
# Model 1B: social distancing, 1% dosage, equal distribution 

infection = []
days = []

num_trials = 10
for each in range(num_trials):  
    medical_len, high_risk_len, essential_len, low_risk_len, day = run_SIR_clusters_2(10,1.5,100000,1.5,[0,0,0,0],[0.25,0.25,0.25,0.25],0.01)
    infection.append(medical_len[1]+high_risk_len[1]+essential_len[1]+low_risk_len[1])
    days.append(day)

LB_infection_2 = np.mean(infection) - scipy.stats.norm.ppf(1-0.05/2)*np.sqrt(np.var(infection)/num_trials) 
UB_infection_2 = np.mean(infection) + scipy.stats.norm.ppf(1-0.05/2)*np.sqrt(np.var(infection)/num_trials)

LB_days_2 = np.mean(days) - scipy.stats.norm.ppf(1-0.05/2)*np.sqrt(np.var(days)/num_trials)
UB_days_2 = np.mean(days) + scipu.stats.norm.ppf(1-0.05/2)*np.sqrt(np.var(days)/num_trials)

print('The 95% CI for the number of infected people is,(', LB_infection,', ', UB_infection, ')')
print('The 95% CI for the number of days it takes to contain the virus is (', LB_days,', ', UB_days, ')')

0.0